# 图表

中国宏观经济数据 [link](https://zhuanlan.zhihu.com/p/30588976)


中国人健康寿命调查 [link](https://www.icpsr.umich.edu/web/NACDA/studies/36692)


中国家庭金融调查 [link](https://chfs.swufe.edu.cn/)


英国有一句老话：“一张图片胜过千言万语。”著名的统计学家John Tukeya，是计算机时代统计图表的先驱者之一。如今，功能强大的软件能够生成各种各样令人眼花缭乱的统计图表，有些是基于网络且可交互的。除了学术研究外，新闻行业也通过许多专业的数据图表吸引了大量的读者。《纽约时报》和《经济学人》就是很好的例子，《南华早报》在这方面也有不错的表现。精美的图表可以直观有力地传达信息。相较于表格，我在撰写论文以及报告时更加青睐图表。因为图表生动而吸引人，表格则单调沉闷。

## 简单的图表

### 时间序列


In [ ]:
library(magrittr)
library(dplyr)
library(doBy)
library(zoo)
library(rvest)
library(Quandl)

实际应用：
- 宏观经济学领域
  - 国家统计局 [National Bureau of Statistics](https://data.stats.gov.cn/easyquery.htm?cn=B01)
  - 美联储 [FRED database](https://research.stlouisfed.org/econ/mccracken/fred-databases/)

- 金融领域
  - 雅虎金融 [Yahoo Finance](https://finance.yahoo.com/)

### 香港 GDP

香港GDP数据 [链接](https://www.censtatd.gov.hk/en/web_table.html?id=33#)

### 美国GDP

美国GDP [经济统计局](https://www.bea.gov/data/gdp/gross-domestic-product)

### 中国 GDP

* 国家统计局 [National Bureau of Statistics](https://data.stats.gov.cn/easyquery.htm?cn=B01)
* 应用编程接口 (Application Programming Interface, API) [FRED's webpage](https://fred.stlouisfed.org/series/MKTGDPCNA646NWDB)

In [ ]:
quantmod::getSymbols.FRED(Symbols = "MKTGDPCNA646NWDB", env = .GlobalEnv) 
plot(MKTGDPCNA646NWDB)

## 上证综合指数

In [ ]:
tick = "000001.SS" # need to find the tick 
SH <- quantmod::getSymbols(tick, auto.assign = FALSE, 
         from = "2000-01-01")[, paste0(tick,".Close")]

plot(SH)

### 000001.SS 收益率

In [ ]:
diff(log(SH)) %>% plot

### 比特币

In [ ]:
BTC <- quantmod::getSymbols("BTC-USD",auto.assign = FALSE, from = "2021-07-01")[,4]
plot(BTC)

ETH <- quantmod::getSymbols("ETH-USD",auto.assign = FALSE, from = "2021-07-01")[,4]
plot(ETH)

plot( x = as.vector(ETH), y = as.vector(BTC), type = "l")

### 比特币收益率

In [ ]:
diff( log(BTC) ) %>% plot( )

### 汇率

In [ ]:
quantmod::getFX("USD/JPY")
quantmod::getFX("HKD/JPY")
matplot( y = cbind(USDJPY, HKDJPY*7.8), x = index(USDJPY), type = "l", xlab = "time"  )

## 图形可视化(GGplot2)

在第一章中，我们已经见过如何用R给OLS回归制作图表。
`plot` 是一个图表制作的通用指令，也是默认的R语言图表引擎。
它能够生成各种功能的统计图表。

许多年来，各国的开发者推出了各种复杂的统计图表程序包。当中最成功的还是 Hadley Wickham 的 
`ggplot2`。

`ggplot2` 一个进阶的画图软件，它能够生成许多高质量的精美图表。它的功能繁多，不可能仅用一章就能介绍完。幸运的是，原作者编写了全面的操作手册
[**ggplot2 book**](http://link.springer.com/book/10.1007%2F978-0-387-98141-3)，通过VPN就能够用港中文的校园网下载下来。


`ggplot2` 支持特定格式的数据框。 `reshape2` 能够帮助我们调整数据为`ggplot2`支持的格式。

ggplot的操作流程是一个一个地在图表中添加元素，然后一起在图表中展示出来。相反， `plot` 先画出主要的图表，之后再补充其他的元素。

`ggplot2`在绘制多个图表方面有其独特的优势，无论是相同格式的图表还是不同格式，它都能起到很好的作用。多个子图表可以呈现更丰富的信息，并有利于我们进行比较。


```{prf:example}

画出两个估计量在三种不同的数据生成过程(DGP)下的密度。这是一个生成相同同样类型图表的例子。
```

In [ ]:
load("data_example/big150.Rdata")
library(ggplot2)
library(reshape2)

big150_1 <- big150[, c("typb", "b1", "b1_c")]
print(head(big150_1))

big150_1 <- melt(big150_1, id.vars = "typb", measure.vars = c("b1", "b1_c"))
names(big150_1)[2] <- c("estimator")
print(head(big150_1))

p1 <- ggplot(big150_1)
p1 <- p1 + geom_area(
  stat = "density", alpha = .25,
  aes(x = value, fill = estimator), position = "identity"
)
p1 <- p1 + facet_grid(. ~ typb)
p1 <- p1 + geom_vline(xintercept = 0)
p1 <- p1 + theme_bw()
p1 <- p1 + theme(
  strip.text = element_text(size = 12),
  axis.text = element_text(size = 12)
)
print(p1)

函数 `ggplot` 选择用哪一个数据集进行绘制；`geom_***`决定图表的形状，例如散点图，折线图、曲线图等。`theme`用于调整细节的元素，比如背景、坐标轴的大小和文字等。


```{prf:example}
这个例子将两个不同类型的图表排版在同一页中。类似的图表也有出现在{cite}`shi2018structural`中。
```

In [ ]:
# graph packages
library(lattice)
library(ggplot2)
library(reshape2)
library(gridExtra)

load("data_example/multigraph.Rdata") # load data


# unify the theme in the two graphs
theme1 <- theme_bw() + theme(
  axis.title.x = element_blank(),
  strip.text = element_text(size = 12),
  axis.text = element_text(size = 12),
  legend.position = "bottom", legend.title = element_blank()
)

# sub-graph 1
d1 <- data.frame(month = 1:480, m = m_vec)
p1 <- qplot(x = month, y = m, data = d1, geom = "line")
p1 <- p1 + theme1 + ylab("fraction of chartists")


# sug-graph 2
d2$month <- 1:480
p2 <- ggplot(d2)
p2 <- p2 + geom_line(aes(x = month, y = value, col = variable))
p2 <- p2 + theme1 + ylab("price and fundamental")

# generate the grahp
grid.arrange(p1, p2, nrow = 2)

为了统一两个图表的主题，我们定义一个主题`theme1`，然后将其应用于两个图表`p1`和`p2`。

## Shiny 交互图表

随着时代的发展，人们越来越重视图表的“交互性”。“交互性”允许用户自行输入自身的需求，从而改变数据输出的形式。Shiny就是一个基于网络(web-based)的构建交互式图表的R包。它的使用可以参考网站[tutorial](https://shiny.rstudio.com/tutorial/)

Shiny的主要组成结构有两个：1.负责显示交互界面的UI 2.负责处理输入信息的服务器(Server)。

### 用户界面

用户界面(User Interface)使用代码来定义Shiny在浏览器中的呈现形式，比如使用滑动条选择柱状图的矩形个数、提供一个输入窗口以输入观测值的个数、各种按钮和标签等等。

### 服务器
服务器用于处理通过UI输入数据。一般来说，服务器是远程运行的程序，它接收来自用户的输入（用户通过交互提出的请求），依照请求提供输出（例如文本、图表）并显示在浏览器中。服务器的处理是**反应式**的，这意味着每一次有新的输入，服务器就会对应重新进行一次新的运算，并将运算结果自动显示。

```{prf:example}
Shiny包内置有11个例子来展示Shiny的功能，我们可以通过`runExample()`来测试这些例子。在文件夹`data_example`中我也放入了一个shiny app的例子，名为`shiny`。
```

In [1]:
install.package("shiny")

ERROR: Error in install.package("shiny"): could not find function "install.package"


In [5]:
library(shiny)
runExample("01_hello")

Warning message:
"package 'shiny' was built under R version 4.1.3"


ERROR: Error: package or namespace load failed for 'shiny' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace 'htmltools' 0.5.2 is already loaded, but >= 0.5.4 is required


In [4]:
runExample("02_text")

ERROR: Error in runExample("02_text"): could not find function "runExample"


In [ ]:
runExample("03_reactivity")

### 可交互图表

在文件夹 `data_example`中，我们提供了一个
`flexdashboard`的简单例子。使用`plotly::ggplotly()`就能很容易将一个GGplot2转化为HTML的可交互图表。



### 后续写作计划

* Shiny app [tutorial](https://shiny.rstudio.com/tutorial/). I have included a shiny app in `data_example/shiny`.